In [1]:
!pip install tk
!pip install pillow
!pip install opencv-python
!pip install deepface

In [8]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from deepface import DeepFace
import numpy as np

def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        global img1, img_pil
        img1 = cv2.imread(file_path)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img1 = Image.fromarray(img1)
        img1.thumbnail((300, 300))  # Resize image to fit in the UI
        img_pil = img1.copy()
        img1 = ImageTk.PhotoImage(img1)
        image_label.config(image=img1)
        image_label.image = img1  # Keep reference to avoid garbage collection
        process_button.config(state=tk.NORMAL)

def update_results_text(results):
    results_text.config(state=tk.NORMAL)
    results_text.delete(1.0, tk.END)  # Clear previous results
    for task, task_result in results.items():
        results_text.insert(tk.END, f"{task.capitalize()}:\n{task_result}\n\n")
    results_text.config(state=tk.DISABLED)

def process_image():
    tasks = []
    if age_var.get():
        tasks.append('age')
    if gender_var.get():
        tasks.append('gender')
    if emotion_var.get():
        tasks.append('emotion')

    img_pil_resized = img_pil.resize((300, 300), Image.ANTIALIAS)
    img_cv = cv2.cvtColor(np.array(img_pil_resized), cv2.COLOR_RGB2BGR)

    result = DeepFace.analyze(img_cv, actions=tasks)

    task_results = {}
    for task in tasks:
        if task == 'emotion':
            task_results[task] = f"{task.capitalize()}: {result[0]['dominant_emotion']}"
        elif task == 'age':
            task_results[task] = f"{task.capitalize()}: {result[0]['age']}"
        elif task == 'gender':
            task_results[task] = f"{task.capitalize()}: {result[0]['gender']}"

    update_results_text(task_results)

root = tk.Tk()
root.title("Facial Analysis App")

upload_button = tk.Button(root, text="Upload Your Image", command=open_image)
upload_button.pack()

image_label = tk.Label(root)
image_label.pack()

age_var = tk.IntVar()
age_checkbox = tk.Checkbutton(root, text="Age", variable=age_var)
age_checkbox.pack()

gender_var = tk.IntVar()
gender_checkbox = tk.Checkbutton(root, text="Gender", variable=gender_var)
gender_checkbox.pack()

emotion_var = tk.IntVar()
emotion_checkbox = tk.Checkbutton(root, text="Emotion", variable=emotion_var)
emotion_checkbox.pack()

process_button = tk.Button(root, text="Process Image", command=process_image, state=tk.DISABLED)
process_button.pack()

results_text = tk.Text(root, height=10, width=70, state=tk.DISABLED)
results_text.pack()

root.mainloop()

Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.41it/s]
